In [33]:
! pip install bert-tensorflow
! pip install tensorflow_text

In [34]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import plot_model
import keras
from bert.tokenization import FullTokenizer
import re
from sklearn.metrics import accuracy_score, classification_report

In [35]:
# Some constants
index_to_tag = {0: '-PAD-', 1: 'ADJ', 2: 'ADP', 3: 'ADV', 4: 'CONJ', 5: 'DET', 6: 'NOUN', 7: 'NUM', 8: 'PRON', 9: 'PRT', 10: 'PUNCT', 11: 'VERB', 12: 'X'}
max_seq_length = 72
checkpoint_path = 'drive/MyDrive/my_best_model_8.hdf5'
n_tags = 13

In [36]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

In [37]:
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)

In [38]:
bert_inputs = [input_word_ids, input_mask, segment_ids]
bert_output = bert_model(bert_inputs)[1]
outputs = keras.layers.Dense(n_tags, activation=keras.activations.softmax)(bert_output)
model = keras.models.Model(inputs=bert_inputs, outputs=outputs)
model.summary(200)

Model: "model_1"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                     Output Shape                                Param #                 Connected to                                                      
 input_word_ids (InputLayer)                                      [(None, 72)]                                0                       []                                                                
                                                                                                                                                                                                        
 input_mask (InputLayer)                                          [(None, 72)]                                0                       []                                           

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
model.load_weights(checkpoint_path)

# Testing model with validation data

In [41]:
! git clone https://github.com/Vishisht-rao/POS-Tagger-Using-Transformers.git

fatal: destination path 'POS-Tagger-Using-Transformers' already exists and is not an empty directory.


In [42]:
val_input_ids = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_input_ids.npz')
val_input_masks = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_input_masks.npz')
val_type_ids = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_type_ids.npz')
val_tags = np.load('POS-Tagger-Using-Transformers/preprocessing_outputs_final/test_tags.npz')

In [43]:
val_input_ids = val_input_ids['arr_0']
val_input_masks = val_input_masks['arr_0']
val_type_ids = val_type_ids['arr_0']
val_tags = val_tags['arr_0']

In [44]:
val_tags_pred = model.predict([val_input_ids, val_input_masks, val_type_ids], batch_size=16)

In [45]:
val_tags_pred_argmax = val_tags_pred.argmax(-1)
val_tags_argmax = val_tags.argmax(-1)

In [46]:
val_tags_pred_argmax[0]

array([ 0, 11,  6,  6,  3, 11,  5,  6,  6,  2,  6,  7,  2,  7, 10,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0])

In [47]:
val_tags_argmax[0]

array([ 0, 11,  6,  6,  3, 11,  5,  6,  6,  2,  6,  7,  2,  7, 10,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0])

In [48]:
val_tags_argmax = val_tags_argmax.flatten()
val_tags_pred_argmax = val_tags_pred_argmax.flatten()

In [49]:
print(classification_report(val_tags_argmax, val_tags_pred_argmax))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    368884
           1       0.94      0.93      0.94     10899
           2       0.97      0.97      0.97     18427
           3       0.95      0.94      0.95      6948
           4       0.98      0.97      0.97      4781
           5       0.98      0.98      0.98     17130
           6       0.96      0.97      0.97     38589
           7       0.96      0.96      0.96      2813
           8       0.97      0.97      0.97      6043
           9       0.94      0.93      0.93      4176
          10       0.97      0.97      0.97     19251
          11       0.98      0.97      0.97     23739
          12       0.92      0.89      0.91       752

    accuracy                           0.99    522432
   macro avg       0.96      0.96      0.96    522432
weighted avg       0.99      0.99      0.99    522432



In [50]:
index_to_tag

{0: '-PAD-',
 1: 'ADJ',
 2: 'ADP',
 3: 'ADV',
 4: 'CONJ',
 5: 'DET',
 6: 'NOUN',
 7: 'NUM',
 8: 'PRON',
 9: 'PRT',
 10: 'PUNCT',
 11: 'VERB',
 12: 'X'}